In [ ]:
#select the mode of panopt

In [ ]:
#load the picture/mask/label

In [1]:
#SIFT
#create the feature point

In [2]:
#use the same label mask to seperate the picture's feature point

In [3]:
#matching each label picture

In [4]:
#summary all the matching into one picture 

In [5]:
#calculate the Homography matrix